<a href="https://colab.research.google.com/github/ChungbinKim/-/blob/master/TextRank2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
import glob
import pickle
import pandas as pd
import numpy as np
import sys
import os
import re
import datetime 
import networkx
import json
from konlpy.tag import Komoran

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 14.6MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
tagger = Komoran()

### 1. 전처리용 클래스 정의

#### 1. A.  원문 텍스트 클린징 + 모델에 문장 할당용

In [3]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

In [4]:
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

#### 1. B. Komoran 형태소 분석기를 활용하여 단어의 품사를 태깅

In [5]:
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

In [6]:
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

### 2. TextRank 클래스 정의

In [7]:
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 4)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.003)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
        
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.4):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

### 3. TextRank 알고리즘을 활용하여 문서를 요약하는 함수 정의

- (참고) 요약 비율은 문서의 20%
- (참고) "휴면"이 포함된 문장은 모두 추출

In [8]:
def summarise_contents(x):
    tr = TextRank()

    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'),('없','VV') ])
    #텍스트랭크 계산에는 들어가지 않지만 출력값에는 들어도 되는 단어
  
    
    tr.loadSents(RawSentence(x), lambda sent: filter(lambda y:y not in stopword and y[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
    tr.build()
    ranks = tr.rank()
    if len(x)>15000:
      a= tr.summarize(0.05)
    elif len(x)>10000:
      a= tr.summarize(0.09)
    else:
      a= tr.summarize(0.15) 
    return a
    #return tr.summarize(0.15)

In [9]:
def extract_resting(x): #여기서 꼭 들어가야 되는 단어 정하기
    
    out = ''
    for line in x.split('\n'):
        if not line.isspace() and '휴면' in line:
            out += line + ' '
    return out.strip()

In [10]:
def extract_resting2(x): #여기서 들어가면 안되는 단어 정하기
  
    out = ''
    for line in x.split('\n'): #텍스트랭크 계산에도 들어가지 않으며 출력결과에도 들어가지 않음
        if not line.isspace() and '준수합니다' not in line and '노력합니다' not in line and '항에' not in line and '각 호' not in line and '말합니다' not in line and '의미합니다' not in line and '각호' not in line and '경우' not in line:
            out += line + ' '
    return out.strip()

In [11]:
def extract_resting3(x): #부칙도 필요하려나
    
    out = ''
    for line in x.split('\n'):
        if not line.isspace() and '부칙' in line:
            out += line + ' '
    return out.strip()

### 4. Data load

#### 4. A. Data load _ Colab ver.

#### Colab

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
file_list = glob.glob('./drive/MyDrive/data/*_raw.txt')
regex_idx = re.compile('[0-9]+')
regex_text = re.compile('[^0-9a-zA-Z가-힣.()\n]+')
dict_ = {}

for file in file_list:
    sentence = ''
    idx = re.findall(regex_idx, file)[0]
    with open(file, 'r') as f:
        for line in f.readlines():
            sentence += line
    dict_[int(idx)] = re.sub(regex_text, ' ', sentence)

#### 4. B. Data load _ Local ver.

#### Local

In [14]:
#file_list = glob.glob('../data/*_raw.txt')
#regex_idx = re.compile('[0-9]+')
#regex_text = re.compile('[^0-9a-zA-Z가-힣.()\n]+')
#dict_ = {}

#for file in file_list:
 #   sentence = ''
  #  idx = re.findall(regex_idx, file)[0]
   # with open(file, 'r') as f:
    #     for line in f.readlines():
     #        sentence += line
    #dict_[int(idx)] = re.sub(regex_text, ' ', sentence)

In [15]:
#glob.glob('../data/*_raw.txt')

In [16]:
data = pd.DataFrame.from_dict(dict_, orient='index', columns=['Text'])

### 5. 문서요약 실행

In [17]:
data['Text2'] = data['Text'].map(lambda x : extract_resting2(x))

data['Text_Rank'] = data['Text2'].map(lambda x: (summarise_contents(x)) if len(x) >1000 else "The sentence is too short to summarize..")
data['Resting'] = data['Text'].map(lambda x :  extract_resting(x))
data['Resting2'] = data['Text'].map(lambda x :  extract_resting3(x))
data['Summary'] = data['Text_Rank'] + data['Resting2']
#data['Summary'] = data['Resting2']

# df.to_csv('sample.csv', index = False) # 결과를 저장
print("Done ! ")

Done ! 


In [18]:
#k = input('몇번 문서를 요약하겠습니까?, 숫자만 쓸것')

In [19]:
#i = int(k)

In [20]:
#data['Text'][i] # Dataframe의 index는 sample 번호와 일치 ,원문

In [21]:
#data['Text2'][i] #원문에서 stopword가 들어간 문장 전체를 제거한 것

In [22]:
data['Summary'][1]#요약문 
#a=data['Summary'][1]# 만약 data['Summary'][1]값이 출력 안될때 
#print(a)


'서비스 이용약관   제1조 목적 본 약관(이하 약관 )은 웰(이하 회사)와 회원에 관한 제반사항을 규정함을 목적으로 합니다.   제9조 회원에 대한 통지 제10조 회사의 의무  회사는 본 약관이 정하는 바에 따라 지속적이고 안정적인 서비스를 제공하는데 최선을 다하여야 합니다.'

In [23]:
b=data['Resting'][1]
print()
print()
print()
print()
bc=b.split()
#print(bc)
list = []
for word in bc:
    if word.endswith('년'):
        list.append(word)
        break
    elif word.endswith('년이상'):
        list.append(word[0:-2])
        break     
    elif word.endswith('개월'):
        list.append(word)
        break
    elif word.endswith('개월이상'):
        list.append(word[0:-2])
        break    
if '삭제' in b:
  dell = 'O'
else:
  dell = 'X'  
if '메일' in b:
  ddell = 'O'
else:
  ddell = 'X' 
#print('    휴면기간       :' , list)
#print('장기휴면시 계정 삭제:',dell)
#print('휴면계정 이메일 통보:',ddell)

col = ['서비스']  # columns 리스트
ind = ['휴면기간', '장기휴면시 계정 삭제', '휴면계정 이메일 통보' ]        # index 리스트
con = [list,dell,ddell]
pd.DataFrame(con, columns=col, index=ind)

,서비스
휴면기간,24개월
장기휴면시 계정 삭제,O
휴면계정 이메일 통보,O
